In [1]:
import pandas as pd
import folium
from folium import plugins
import numpy as np
import random
import branca.colormap as cm

# =============================================================================
# 1. CARGA DE DATOS
# =============================================================================

print("--- INICIANDO ANÁLISIS ESPACIAL AVANZADO (DASHBOARD GEOGRÁFICO) ---")

archivo_datos = 'WA_Fn-UseC_-Telco-Customer-Churn.csv'

try:
    df = pd.read_csv(archivo_datos)
    print(f"✅ Dataset cargado: {len(df)} clientes.")
except FileNotFoundError:
    print("⚠️ Archivo no encontrado. Usando datos simulados para demostración.")
    # Datos dummy para demostración
    df = pd.DataFrame({
        'customerID': [f'ID-{i}' for i in range(100)],
        'Churn': np.random.choice(['Yes', 'No'], 100),
        'MonthlyCharges': np.random.uniform(50, 120, 100),
        'TotalCharges': np.random.uniform(1000, 5000, 100),
        'InternetService': np.random.choice(['Fiber optic', 'DSL'], 100),
        'Contract': np.random.choice(['Month-to-month', 'Two year'], 100),
        'tenure': np.random.randint(1, 72, 100),
        'PaymentMethod': np.random.choice(['Electronic check', 'Mailed check'], 100)
    })

# Convertir Churn a numérico
df['Churn_Num'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)

# =============================================================================
# 2. GENERACIÓN DE COORDENADAS (SIMULACIÓN INTELIGENTE)
# =============================================================================

print("📍 Generando contexto geográfico (California)...")

if 'Latitude' not in df.columns:
    # Centros urbanos principales de California
    centros = [
        (34.0522, -118.2437, 'Los Angeles'),
        (37.7749, -122.4194, 'San Francisco'),
        (32.7157, -117.1611, 'San Diego'),
        (38.5816, -121.4944, 'Sacramento'),
        (36.7378, -119.7871, 'Fresno')
    ]

    def generar_lat_lon_zona():
        centro = random.choice(centros)
        # Dispersión aleatoria alrededor de la ciudad
        lat = centro[0] + np.random.normal(0, 0.08)
        lon = centro[1] + np.random.normal(0, 0.08)
        return lat, lon, centro[2]

    coords = [generar_lat_lon_zona() for _ in range(len(df))]
    df['Latitude'], df['Longitude'], df['Zona'] = zip(*coords)

# =============================================================================
# 3. CONFIGURACIÓN DEL MAPA
# =============================================================================

mapa = folium.Map(location=[36.7783, -119.4179], zoom_start=6, tiles='CartoDB dark_matter')

# =============================================================================
# 4. CAPA 1: HEATMAP DE RIESGO
# =============================================================================
print("🔥 Generando Heatmap de Ingresos en Riesgo...")

df_churn = df[df['Churn'] == 'Yes'].copy()
# Ponderamos por MonthlyCharges para ver dónde perdemos DINERO, no solo gente
heat_data = df_churn[['Latitude', 'Longitude', 'MonthlyCharges']].values.tolist()

plugins.HeatMap(
    heat_data,
    name='🔥 Zonas de Pérdida ($)',
    radius=15,
    blur=10,
    gradient={0.4: 'purple', 0.65: 'orange', 1: 'red'}
).add_to(mapa)

# =============================================================================
# 5. CAPA 2: MARCADORES CON "CUADRO DE INFORMACIÓN" (POPUP)
# =============================================================================
print("📊 Construyendo cuadros de información detallada...")

# Creamos Cluster
marker_cluster = plugins.MarkerCluster(name='👥 Detalle de Clientes').add_to(mapa)

# Muestra (limitada para rendimiento en navegador, 500 puntos)
df_sample = df.sample(min(len(df), 500))

for i, row in df_sample.iterrows():
    # Lógica de Color e Ícono
    es_churn = row['Churn'] == 'Yes'
    color = 'red' if es_churn else 'green'
    icono_nombre = 'exclamation-triangle' if es_churn else 'check-circle'
    estado_texto = "CANCELADO" if es_churn else "ACTIVO"

    # --- CUADRO DE INFORMACIÓN HTML (ESTILO DASHBOARD) ---
    # Usamos CSS inline para crear una mini tarjeta profesional
    popup_html = f"""
    <div style="
        font-family: 'Segoe UI', Arial, sans-serif;
        width: 280px;
        background-color: #f9f9f9;
        border: 1px solid #ccc;
        border-radius: 8px;
        box-shadow: 2px 2px 5px rgba(0,0,0,0.2);
        overflow: hidden;
    ">
        <div style="
            background-color: {color};
            color: white;
            padding: 10px;
            font-weight: bold;
            font-size: 14px;
            text-align: center;
        ">
            CLIENTE: {row['customerID']} <br>
            <span style="font-size:10px">ESTADO: {estado_texto}</span>
        </div>

        <div style="padding: 15px;">
            <table style="width:100%; font-size:12px; border-collapse: collapse;">
                <tr style="border-bottom: 1px solid #eee;">
                    <td style="padding: 5px; color: #555;"><b>Zona:</b></td>
                    <td style="padding: 5px; text-align:right;">{row['Zona']}</td>
                </tr>
                <tr style="border-bottom: 1px solid #eee;">
                    <td style="padding: 5px; color: #555;"><b>Cargo Mensual:</b></td>
                    <td style="padding: 5px; text-align:right;"><b>${row['MonthlyCharges']:.2f}</b></td>
                </tr>
                 <tr style="border-bottom: 1px solid #eee;">
                    <td style="padding: 5px; color: #555;"><b>Total Facturado:</b></td>
                    <td style="padding: 5px; text-align:right;">${row['TotalCharges']}</td>
                </tr>
                <tr style="border-bottom: 1px solid #eee;">
                    <td style="padding: 5px; color: #555;"><b>Servicio Internet:</b></td>
                    <td style="padding: 5px; text-align:right;">{row['InternetService']}</td>
                </tr>
                <tr>
                    <td style="padding: 5px; color: #555;"><b>Contrato:</b></td>
                    <td style="padding: 5px; text-align:right;">{row['Contract']}</td>
                </tr>
            </table>

            <div style="margin-top: 10px; font-size: 11px; color: #777; text-align: center;">
                Antigüedad: {row['tenure']} meses
                <div style="background-color: #e0e0e0; height: 4px; border-radius: 2px; margin-top: 2px; width: 100%;">
                    <div style="background-color: #333; height: 100%; width: {min(row['tenure'], 100)}%;"></div>
                </div>
            </div>
        </div>
    </div>
    """

    # Creamos el IFrame para que el HTML complejo se renderice bien
    iframe = folium.IFrame(popup_html, width=300, height=260)
    popup = folium.Popup(iframe, max_width=300)

    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup,
        icon=folium.Icon(color=color, icon=icono_nombre, prefix='fa'),
        tooltip=f"ID: {row['customerID']} (${row['MonthlyCharges']})"
    ).add_to(marker_cluster)

# =============================================================================
# 6. EXPORTACIÓN
# =============================================================================

folium.LayerControl(collapsed=False).add_to(mapa)
plugins.Fullscreen().add_to(mapa)

salida = 'mapa_telco_dashboard_completo.html'
mapa.save(salida)

print(f"\n✅ ¡DASHBOARD GEOGRÁFICO GENERADO!: {salida}")
print("   Características Nuevas:")
print("   1. Tarjetas de Información (Popups) con diseño HTML/CSS profesional.")
print("   2. Tablas de datos financieros por cliente.")
print("   3. Barra de progreso visual para la antigüedad.")

--- INICIANDO ANÁLISIS ESPACIAL AVANZADO (DASHBOARD GEOGRÁFICO) ---
⚠️ Archivo no encontrado. Usando datos simulados para demostración.
📍 Generando contexto geográfico (California)...
🔥 Generando Heatmap de Ingresos en Riesgo...
📊 Construyendo cuadros de información detallada...

✅ ¡DASHBOARD GEOGRÁFICO GENERADO!: mapa_telco_dashboard_completo.html
   Características Nuevas:
   1. Tarjetas de Información (Popups) con diseño HTML/CSS profesional.
   2. Tablas de datos financieros por cliente.
   3. Barra de progreso visual para la antigüedad.
